In [ ]:
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq
!pip install -U --no-build-isolation --no-deps ../input/hf-transformers -qq
!pip install -U --no-build-isolation --no-deps ../input/opentamil/Open-Tamil-1.0/Open-Tamil-1.0 -qq

In [ ]:
import gc
gc.enable()
import torch
import collections
import numpy as np
import transformers
import pandas as pd
from torch import nn
from tqdm.auto import tqdm
from datasets import Dataset
from functools import partial
from torch.nn import CrossEntropyLoss
from tamilstemmer import TamilStemmer
from transformers import default_data_collator
from transformers import AutoTokenizer, Trainer, BertForQuestionAnswering
from transformers import XLMRobertaForQuestionAnswering, RemBertForQuestionAnswering, TrainingArguments
from transformers.modeling_outputs import QuestionAnsweringModelOutput

punctuation = '!"#$&\'()*+,-/:;<=>?@[\\]^_`{|}~'

In [ ]:
class CustomXLMRobertaForQuestionAnswering(XLMRobertaForQuestionAnswering):
    def __init__(self, config):
        super().__init__(config)
        self.end_outputs = nn.Linear(config.hidden_size, 1)
        self.start_outputs = nn.Linear(config.hidden_size+1, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        end_logits = self.end_outputs(sequence_output)
        start_logits_ = torch.cat((sequence_output, end_logits), dim=-1)
        start_logits = self.start_outputs(start_logits_)
        
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        total_loss = None
        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
class CustomBERTForQuestionAnswering(BertForQuestionAnswering):
    def __init__(self, config):
        super().__init__(config)
        self.end_outputs = nn.Linear(config.hidden_size, 1)
        self.start_outputs = nn.Linear(config.hidden_size+1, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )

        sequence_output = outputs[0]

        end_logits = self.end_outputs(sequence_output)
        start_logits_ = torch.cat((sequence_output, end_logits), dim=-1)
        start_logits = self.start_outputs(start_logits_)
        
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        total_loss = None

        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
class CustomRemBertForQuestionAnswering(RemBertForQuestionAnswering):
    def __init__(self, config):
        super().__init__(config)
        self.end_outputs = nn.Linear(config.hidden_size, 1)
        self.start_outputs = nn.Linear(config.hidden_size+1, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.rembert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        end_logits = self.end_outputs(sequence_output)
        start_logits_ = torch.cat((sequence_output, end_logits), dim=-1)
        start_logits = self.start_outputs(start_logits_)
        
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        total_loss = None

        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
XLM_ROBERTA_DATASETS = ["xlmrobertalargewithsquadv2tydiqasqdtrans384f"]
XLM_ROBERTA_CLASSES = [CustomXLMRobertaForQuestionAnswering]
XLM_ROBERTA_WEIGHT = [1]

Muril_DATASETS = ["murillargecasedchaii"]
Muril_CLASSES = [CustomBERTForQuestionAnswering]
Muril_WEIGHT = [1]

RemBERT_DATASETS = ["modelsrembertwithsquadv2tydiqa384"]
RemBERT_CLASSES = [CustomRemBertForQuestionAnswering]
RemBERT_WEIGHT = [1]

DO_SIGMOID = False

In [ ]:
def convert_answers(r):
    start = r[0]
    text = r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

In [ ]:
def prepare_validation_features(examples, tokenizer, MAX_LENGTH, DOC_STRIDE):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]
    
    pad_on_right = tokenizer.padding_side == "right"
    
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
def CustomSoftmax(examples, features, raw_predictions):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]
        start_logits_sum = 0
        end_logits_sum = 0
        
        logCs = None
        logCe = None
        for feature_index in feature_indices:
            if logCs is None:
                logCs = all_start_logits[feature_index].max()
            else:
                logCs = max(logCs, all_start_logits[feature_index].max())
            
            if logCe is None:
                logCe = all_end_logits[feature_index].max()
            else:
                logCe = max(logCe, all_end_logits[feature_index].max())
        
        for feature_index in feature_indices:
            all_start_logits[feature_index] -= logCs
            all_end_logits[feature_index] -= logCe
            start_logits_sum += np.exp(all_start_logits[feature_index]).sum()
            end_logits_sum += np.exp(all_end_logits[feature_index]).sum()
            
        for feature_index in feature_indices:
            all_start_logits[feature_index] = np.exp(all_start_logits[feature_index])/start_logits_sum
            all_end_logits[feature_index] = np.exp(all_end_logits[feature_index])/end_logits_sum
        
    return all_start_logits, all_end_logits

In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, tokenizer, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()
    scores = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    all_start_logits_softmax, all_end_logits_sofmax = CustomSoftmax(examples, features, (all_start_logits.copy(), all_end_logits.copy()))
    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            start_logits_softmax = all_start_logits_softmax[feature_index]
            end_logits_softmax = all_end_logits_sofmax[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char],
                            "start_logits": torch.tensor(start_logits_softmax[start_index]),
                            "end_logits": torch.tensor(end_logits_softmax[end_index])
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": context, "score": 0.0, "start_logits": 0.0, "end_logits": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]
        scores[example["id"]] = best_answer["start_logits"]+best_answer["end_logits"]

    return predictions, scores

In [ ]:
def get_predictions(model_checkpoint, feats, ModelForQuestionAnswering, tokenizer):
    model = ModelForQuestionAnswering.from_pretrained(model_checkpoint)
    args = TrainingArguments(
        "tmp",
        dataloader_num_workers=4,
        per_device_eval_batch_size=128,
        report_to="none"
    )
    trainer = Trainer(
        model,
        args,
        data_collator=default_data_collator,
        tokenizer=tokenizer,
    )
    test_predictions = trainer.predict(feats)
    del model, tokenizer, trainer
    gc.collect()
    
    start_logits, end_logits = test_predictions.predictions
    if DO_SIGMOID:
        start_logits = torch.from_numpy(start_logits).sigmoid().numpy()
        end_logits = torch.from_numpy(end_logits).sigmoid().numpy()

    return start_logits, end_logits

In [ ]:
def jaccard(row):
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def union(row):
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.union(b)
    return " ".join(c)

In [ ]:
tamil_ad = "கி.பி"
tamil_bc = "கி.மு"
tamil_km = "கி.மீ"
hindi_ad = "ई"
hindi_bc = "ई.पू"

In [ ]:
def MinMaxScaler(X, target_min, target_max):
    X = (X - X.min()) / (X.max() - X.min())
    return X * (target_max - target_min) + target_min

In [ ]:
def shrink201(X):
    X = (X - X.min(-1, keepdims=True))/(X.max(-1, keepdims=True) - X.min(-1, keepdims=True))
    return X

In [ ]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test_dataset = Dataset.from_pandas(test)

In [ ]:
## XLM ROBERTA

start_logits = 0
end_logits = 0

XLM_ROBERTA_tokenizer = AutoTokenizer.from_pretrained("../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2")

XLM_ROBERTA_prepare_validation_features = partial(prepare_validation_features, tokenizer=XLM_ROBERTA_tokenizer, MAX_LENGTH=384, DOC_STRIDE=128)


XLM_ROBERTA_test_features = test_dataset.map(
    XLM_ROBERTA_prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)
XLM_ROBERTA_test_feats_small = XLM_ROBERTA_test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
XLM_ROBERTA_test_features.set_format(type=XLM_ROBERTA_test_features.format["type"], columns=list(XLM_ROBERTA_test_features.features.keys()))

for i in range(len(XLM_ROBERTA_DATASETS)):

    start_logits1, end_logits1 = get_predictions(f'../input/{XLM_ROBERTA_DATASETS[i]}/FOLD0', XLM_ROBERTA_test_features, XLM_ROBERTA_CLASSES[i], XLM_ROBERTA_tokenizer)
    start_logits2, end_logits2 = get_predictions(f'../input/{XLM_ROBERTA_DATASETS[i]}/FOLD1', XLM_ROBERTA_test_features, XLM_ROBERTA_CLASSES[i], XLM_ROBERTA_tokenizer)
    start_logits3, end_logits3 = get_predictions(f'../input/{XLM_ROBERTA_DATASETS[i]}/FOLD2', XLM_ROBERTA_test_features, XLM_ROBERTA_CLASSES[i], XLM_ROBERTA_tokenizer)
    start_logits4, end_logits4 = get_predictions(f'../input/{XLM_ROBERTA_DATASETS[i]}/FOLD3', XLM_ROBERTA_test_features, XLM_ROBERTA_CLASSES[i], XLM_ROBERTA_tokenizer)
    start_logits5, end_logits5 = get_predictions(f'../input/{XLM_ROBERTA_DATASETS[i]}/FOLD4', XLM_ROBERTA_test_features, XLM_ROBERTA_CLASSES[i], XLM_ROBERTA_tokenizer)
    
    start_logits += XLM_ROBERTA_WEIGHT[i]*(start_logits1 + start_logits2 + start_logits3 + start_logits4 + start_logits5) / 5
    end_logits += XLM_ROBERTA_WEIGHT[i]*(end_logits1 + end_logits2 + end_logits3 + end_logits4 + end_logits5) / 5

start_logits/=sum(XLM_ROBERTA_WEIGHT)
end_logits/=sum(XLM_ROBERTA_WEIGHT)

final_predictions, final_scores = postprocess_qa_predictions(test_dataset, XLM_ROBERTA_test_features, (start_logits, end_logits), tokenizer=XLM_ROBERTA_tokenizer)
references = [{"id": ex["id"]} for ex in test_dataset]
res = pd.DataFrame(references)
res['XLM_ROBERTA_score'] = res['id'].apply(lambda r: final_scores[r].item())
res['XLM_ROBERTA_prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['XLM_ROBERTA_prediction'] = res['XLM_ROBERTA_prediction'].str.replace(r'[!\$&\()*+\=#@?\[\\\]^_`{|}~]*', '')
res['XLM_ROBERTA_prediction'] = res['XLM_ROBERTA_prediction'].apply(lambda x: " ".join(x.split()))
res['XLM_ROBERTA_prediction'] = res['XLM_ROBERTA_prediction'].str.strip(punctuation)

cleaned_preds = []
for pred, context in zip(res["XLM_ROBERTA_prediction"], test["context"]):

    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."

    cleaned_preds.append(pred)

res["XLM_ROBERTA_prediction"] = cleaned_preds

In [ ]:
## Muril

start_logits = 0
end_logits = 0

Muril_tokenizer = AutoTokenizer.from_pretrained(f'../input/{Muril_DATASETS[0]}/FOLD0')

Muril_prepare_validation_features = partial(prepare_validation_features, tokenizer=Muril_tokenizer, MAX_LENGTH=384, DOC_STRIDE=128)


Muril_test_features = test_dataset.map(
    Muril_prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)
Muril_test_feats_small = Muril_test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
Muril_test_features.set_format(type=Muril_test_features.format["type"], columns=list(Muril_test_features.features.keys()))

for i in range(len(Muril_DATASETS)):

    start_logits1, end_logits1 = get_predictions(f'../input/{Muril_DATASETS[i]}/FOLD0', Muril_test_features, Muril_CLASSES[i], Muril_tokenizer)
    start_logits2, end_logits2 = get_predictions(f'../input/{Muril_DATASETS[i]}/FOLD1', Muril_test_features, Muril_CLASSES[i], Muril_tokenizer)
    start_logits3, end_logits3 = get_predictions(f'../input/{Muril_DATASETS[i]}/FOLD2', Muril_test_features, Muril_CLASSES[i], Muril_tokenizer)
    start_logits4, end_logits4 = get_predictions(f'../input/{Muril_DATASETS[i]}/FOLD3', Muril_test_features, Muril_CLASSES[i], Muril_tokenizer)
    start_logits5, end_logits5 = get_predictions(f'../input/{Muril_DATASETS[i]}/FOLD4', Muril_test_features, Muril_CLASSES[i], Muril_tokenizer)
    
    start_logits += Muril_WEIGHT[i]*(start_logits1 + start_logits2 + start_logits3 + start_logits4 + start_logits5) / 5
    end_logits += Muril_WEIGHT[i]*(end_logits1 + end_logits2 + end_logits3 + end_logits4 + end_logits5) / 5

start_logits/=sum(Muril_WEIGHT)
end_logits/=sum(Muril_WEIGHT)


final_predictions, final_scores = postprocess_qa_predictions(test_dataset, Muril_test_features, (start_logits, end_logits), tokenizer=Muril_tokenizer)
res['Muril_score'] = res['id'].apply(lambda r: final_scores[r].item())
res['Muril_prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['Muril_prediction'] = res['Muril_prediction'].str.replace(r'[!\$&\()*+\=#@?\[\\\]^_`{|}~]*', '')
res['Muril_prediction'] = res['Muril_prediction'].apply(lambda x: " ".join(x.split()))
res['Muril_prediction'] = res['Muril_prediction'].str.strip(punctuation)

cleaned_preds = []
for pred, context in zip(res["Muril_prediction"], test["context"]):
    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."

    cleaned_preds.append(pred)

res["Muril_prediction"] = cleaned_preds

In [ ]:
## RemBERT

start_logits = 0
end_logits = 0

RemBERT_tokenizer = AutoTokenizer.from_pretrained(f'../input/{RemBERT_DATASETS[0]}/FOLD0')

RemBERT_prepare_validation_features = partial(prepare_validation_features, tokenizer=RemBERT_tokenizer, MAX_LENGTH=384, DOC_STRIDE=128)


RemBERT_test_features = test_dataset.map(
    RemBERT_prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)
RemBERT_test_feats_small = RemBERT_test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
RemBERT_test_features.set_format(type=RemBERT_test_features.format["type"], columns=list(RemBERT_test_features.features.keys()))

for i in range(len(RemBERT_DATASETS)):

    start_logits1, end_logits1 = get_predictions(f'../input/{RemBERT_DATASETS[i]}/FOLD0', RemBERT_test_features, RemBERT_CLASSES[i], RemBERT_tokenizer)
    start_logits2, end_logits2 = get_predictions(f'../input/{RemBERT_DATASETS[i]}/FOLD1', RemBERT_test_features, RemBERT_CLASSES[i], RemBERT_tokenizer)
    start_logits3, end_logits3 = get_predictions(f'../input/{RemBERT_DATASETS[i]}/FOLD2', RemBERT_test_features, RemBERT_CLASSES[i], RemBERT_tokenizer)
    start_logits4, end_logits4 = get_predictions(f'../input/{RemBERT_DATASETS[i]}/FOLD3', RemBERT_test_features, RemBERT_CLASSES[i], RemBERT_tokenizer)
    start_logits5, end_logits5 = get_predictions(f'../input/{RemBERT_DATASETS[i]}/FOLD4', RemBERT_test_features, RemBERT_CLASSES[i], RemBERT_tokenizer)
    
    start_logits += RemBERT_WEIGHT[i]*(start_logits1 + start_logits2 + start_logits3 + start_logits4 + start_logits5) / 5
    end_logits += RemBERT_WEIGHT[i]*(end_logits1 + end_logits2 + end_logits3 + end_logits4 + end_logits5) / 5

start_logits/=sum(RemBERT_WEIGHT)
end_logits/=sum(RemBERT_WEIGHT)

final_predictions, final_scores = postprocess_qa_predictions(test_dataset, RemBERT_test_features, (start_logits, end_logits), tokenizer=RemBERT_tokenizer)
res['RemBERT_score'] = res['id'].apply(lambda r: final_scores[r].item())
res['RemBERT_prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['RemBERT_prediction'] = res['RemBERT_prediction'].str.replace(r'[!\$&\()*+\=#@?\[\\\]^_`{|}~]*', '')
res['RemBERT_prediction'] = res['RemBERT_prediction'].apply(lambda x: " ".join(x.split()))
res['RemBERT_prediction'] = res['RemBERT_prediction'].str.strip(punctuation)

cleaned_preds = []
for pred, context in zip(res["RemBERT_prediction"], test["context"]):
    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."

    cleaned_preds.append(pred)

res["RemBERT_prediction"] = cleaned_preds

In [ ]:
def subSelector(x):
    if x.XLM_ROBERTA_score>x.Muril_score and x.XLM_ROBERTA_score>x.RemBERT_score:
        return x.XLM_ROBERTA_prediction
    elif x.Muril_score>x.RemBERT_score:
        return x.Muril_prediction
    else:
        return x.RemBERT_prediction

In [ ]:
res["prediction"] = res.T.apply(lambda x: subSelector(x))

In [ ]:
res

In [ ]:
def ta_stemmer(x):
    if x.language == "tamil":
        xs = TamilStemmer().stemWord(x.predictions)
        if xs in x.context.split():
            x.predictions = xs
    return x.predictions

In [ ]:
sub = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
sub['PredictionString'] = res["prediction"]

In [ ]:
test["predictions"] = sub['PredictionString']
sub['PredictionString'] = test.T.apply(lambda x: ta_stemmer(x))

In [ ]:
sub.to_csv('submission.csv', index=False)